In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-tgq3xp4v
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-tgq3xp4v
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=fccf509e21299065a3760ce568224983c9db541f1ad288f64e5a0acb53ed3a06
  Stored in directory: /tmp/pip-ephem-wheel-cache-hwy4qdv0/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


**Load the Dataset and unzip it**

In [ ]:
!rm -r *
!wget -q https://www.dropbox.com/s/2l0mikg5aupzr1d/Lab2.zip
!unzip Lab2.zip
!rm Lab2.zip
!ls

Archive:  Lab2.zip
  inflating: Dataset/VectorAdd/Test/0/input0.raw  
  inflating: Dataset/VectorAdd/Test/0/input1.raw  
  inflating: Dataset/VectorAdd/Test/0/myOutput.raw  
  inflating: Dataset/VectorAdd/Test/0/output.raw  
  inflating: Dataset/VectorAdd/Test/1/input0.raw  
  inflating: Dataset/VectorAdd/Test/1/input1.raw  
  inflating: Dataset/VectorAdd/Test/1/myOutput.raw  
  inflating: Dataset/VectorAdd/Test/1/output.raw  
  inflating: Dataset/VectorAdd/Test/2/input0.raw  
  inflating: Dataset/VectorAdd/Test/2/input1.raw  
  inflating: Dataset/VectorAdd/Test/2/myOutput.raw  
  inflating: Dataset/VectorAdd/Test/2/output.raw  
  inflating: Dataset/VectorAdd/Test/3/input0.raw  
  inflating: Dataset/VectorAdd/Test/3/input1.raw  
  inflating: Dataset/VectorAdd/Test/3/myOutput.raw  
  inflating: Dataset/VectorAdd/Test/3/output.raw  
  inflating: Dataset/VectorAdd/Test/4/input0.raw  
  inflating: Dataset/VectorAdd/Test/4/input1.raw  
  inflating: Dataset/VectorAdd/Test/4/myOutput.raw  
  

# **OpenCL Vector Add Code**

Code to be modified where there are TODO indications

In [ ]:
%%writefile lab10-OPENCL.cpp
#define _CRT_SECURE_NO_WARNINGS
#define CL_USE_DEPRECATED_OPENCL_1_2_APIS
#include <CL/opencl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

#define wbCheck(ans) { gpuAssert((ans), __FILE__, __LINE__,1); }
float* ImportRawFloat(char* filename, int* N)
{
	FILE* handle;
	float* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return NULL;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return NULL;
	}
	fscanf(handle, "%d", N);
	val = (float*)malloc(*N * sizeof(float));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%f", val + i);
	}
	fclose(handle);
	return val;

}
int ExportRawFloat(char* filename, float* val, int N)
{

	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}

	handle = fopen(filename, "w");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return 0;
	}
	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%f\n", val[i]);
	}
	fclose(handle);
	return 1;
}

const char* getErrorString(cl_int error)
{
	switch (error) {
		// run-time and JIT compiler errors
	case 0: return "CL_SUCCESS";
	case -1: return "CL_DEVICE_NOT_FOUND";
	case -2: return "CL_DEVICE_NOT_AVAILABLE";
	case -3: return "CL_COMPILER_NOT_AVAILABLE";
	case -4: return "CL_MEM_OBJECT_ALLOCATION_FAILURE";
	case -5: return "CL_OUT_OF_RESOURCES";
	case -6: return "CL_OUT_OF_HOST_MEMORY";
	case -7: return "CL_PROFILING_INFO_NOT_AVAILABLE";
	case -8: return "CL_MEM_COPY_OVERLAP";
	case -9: return "CL_IMAGE_FORMAT_MISMATCH";
	case -10: return "CL_IMAGE_FORMAT_NOT_SUPPORTED";
	case -11: return "CL_BUILD_PROGRAM_FAILURE";
	case -12: return "CL_MAP_FAILURE";
	case -13: return "CL_MISALIGNED_SUB_BUFFER_OFFSET";
	case -14: return "CL_EXEC_STATUS_ERROR_FOR_EVENTS_IN_WAIT_LIST";
	case -15: return "CL_COMPILE_PROGRAM_FAILURE";
	case -16: return "CL_LINKER_NOT_AVAILABLE";
	case -17: return "CL_LINK_PROGRAM_FAILURE";
	case -18: return "CL_DEVICE_PARTITION_FAILED";
	case -19: return "CL_KERNEL_ARG_INFO_NOT_AVAILABLE";

		// compile-time errors
	case -30: return "CL_INVALID_VALUE";
	case -31: return "CL_INVALID_DEVICE_TYPE";
	case -32: return "CL_INVALID_PLATFORM";
	case -33: return "CL_INVALID_DEVICE";
	case -34: return "CL_INVALID_CONTEXT";
	case -35: return "CL_INVALID_QUEUE_PROPERTIES";
	case -36: return "CL_INVALID_COMMAND_QUEUE";
	case -37: return "CL_INVALID_HOST_PTR";
	case -38: return "CL_INVALID_MEM_OBJECT";
	case -39: return "CL_INVALID_IMAGE_FORMAT_DESCRIPTOR";
	case -40: return "CL_INVALID_IMAGE_SIZE";
	case -41: return "CL_INVALID_SAMPLER";
	case -42: return "CL_INVALID_BINARY";
	case -43: return "CL_INVALID_BUILD_OPTIONS";
	case -44: return "CL_INVALID_PROGRAM";
	case -45: return "CL_INVALID_PROGRAM_EXECUTABLE";
	case -46: return "CL_INVALID_KERNEL_NAME";
	case -47: return "CL_INVALID_KERNEL_DEFINITION";
	case -48: return "CL_INVALID_KERNEL";
	case -49: return "CL_INVALID_ARG_INDEX";
	case -50: return "CL_INVALID_ARG_VALUE";
	case -51: return "CL_INVALID_ARG_SIZE";
	case -52: return "CL_INVALID_KERNEL_ARGS";
	case -53: return "CL_INVALID_WORK_DIMENSION";
	case -54: return "CL_INVALID_WORK_GROUP_SIZE";
	case -55: return "CL_INVALID_WORK_ITEM_SIZE";
	case -56: return "CL_INVALID_GLOBAL_OFFSET";
	case -57: return "CL_INVALID_EVENT_WAIT_LIST";
	case -58: return "CL_INVALID_EVENT";
	case -59: return "CL_INVALID_OPERATION";
	case -60: return "CL_INVALID_GL_OBJECT";
	case -61: return "CL_INVALID_BUFFER_SIZE";
	case -62: return "CL_INVALID_MIP_LEVEL";
	case -63: return "CL_INVALID_GLOBAL_WORK_SIZE";
	case -64: return "CL_INVALID_PROPERTY";
	case -65: return "CL_INVALID_IMAGE_DESCRIPTOR";
	case -66: return "CL_INVALID_COMPILER_OPTIONS";
	case -67: return "CL_INVALID_LINKER_OPTIONS";
	case -68: return "CL_INVALID_DEVICE_PARTITION_COUNT";

		// extension errors
	case -1000: return "CL_INVALID_GL_SHAREGROUP_REFERENCE_KHR";
	case -1001: return "CL_PLATFORM_NOT_FOUND_KHR";
	case -1002: return "CL_INVALID_D3D10_DEVICE_KHR";
	case -1003: return "CL_INVALID_D3D10_RESOURCE_KHR";
	case -1004: return "CL_D3D10_RESOURCE_ALREADY_ACQUIRED_KHR";
	case -1005: return "CL_D3D10_RESOURCE_NOT_ACQUIRED_KHR";
	default: return "Unknown OpenCL error";
	}
}


inline void gpuAssert(cl_int code, const char *file, int line, int abort)
{
	if (code != CL_SUCCESS)
	{
		printf( "GPUassert: %s %s %d\n", getErrorString(code), file, line);
		if (abort==1) {

			exit(code);
		}
	}
}

// TODO: Add vector add kernel as text
const char* kernelSource =
"\n"
"__kernel void vecAdd(  __global float *a,                       \n"
"                       __global float *b,                       \n"
"                       __global float *c,                       \n"
"                       const unsigned int n)                    \n"
"{                                                               \n"
"    //Get our global thread ID                                  \n"
"    //Make sure we do not go out of bounds                      \n"
"		 int id = get_global_id(0);																	 \n"
"    if (id < n)                                                 \n"
"      c[id] = a[id] + b[id];                                    \n"
"}                                                               \n"
"\n";


clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

int main(int argc, char *argv[]) {
	int i,prt=0;
	int inputLength;
	int inputLengthBytes;
	float* hostInput1;
	float* hostInput2;
	float* hostOutput;
	float* hostExpected;
	cl_mem deviceInput1;
	cl_mem deviceInput2;
	cl_mem deviceOutput;
  float meanDiff = 0;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[50],argv5[3];

	cl_platform_id cpPlatform; // OpenCL platform
	cl_device_id device_id;    // device ID
	cl_context context;        // context
	cl_command_queue queue;    // command queue
	cl_program program;        // program
	cl_kernel kernel;          // kernel

	char* programLog; size_t logSize;

for (ti=0;ti<10;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/VectorAdd/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/VectorAdd/Test/%i/input0.raw",ti);
	sprintf(argv3,"Dataset/VectorAdd/Test/%i/input1.raw",ti);
	sprintf(argv4,"Dataset/VectorAdd/Test/%i/Myoutput.raw",ti);
	sprintf(argv5,"ON");

	StartingTime = Time_start();
	hostExpected = (float*)ImportRawFloat(argv1, &inputLength);
	hostInput1 = (float*)ImportRawFloat(argv2, &inputLength);
	hostInput2 = (float*)ImportRawFloat(argv3, &inputLength);

	if (strcmp(argv5, "ON") == 0)prt = 1;

	printf("The input length is %d\n", inputLength);
	inputLengthBytes = inputLength * sizeof(float);
	hostOutput = (float*)malloc(inputLengthBytes);
	Elapsed_time(StartingTime, "Importing data and creating memory on host", prt);
	// TODO: Insert code here
	// Initialize the workgroup dimensions
	size_t globalSize, localSize;
	cl_int err;
	cl_build_status status;
	//@@ Insert code here
	localSize = 64;
	globalSize = ceil(inputLength / (float)localSize) * localSize;

	// Bind to platform
	err = clGetPlatformIDs(1, &cpPlatform, NULL);
	wbCheck(err);

	// Get ID for the device
	err =clGetDeviceIDs(cpPlatform, CL_DEVICE_TYPE_GPU, 1, &device_id, NULL);
	wbCheck(err);

	// Create a context
	context = clCreateContext(0, 1, &device_id, NULL, NULL, &err);
	wbCheck(err);

	// Create a command queue
	queue = clCreateCommandQueue(context, device_id, 0, &err);
	wbCheck(err);

	// Create the compute program from the source buffer
	program = clCreateProgramWithSource(
		context, 1, (const char**)&kernelSource, NULL, &err);
	wbCheck(err);

	// Build the program executable
	const char options[] = "-Werror -cl-std=CL1.1";
	err = clBuildProgram(program, 0, NULL, NULL, NULL, NULL);

	// build failed
	if (err != CL_SUCCESS) {

		// check build error and build status first
		clGetProgramBuildInfo(program, device_id, CL_PROGRAM_BUILD_STATUS,
			sizeof(cl_build_status), &status, NULL);

		// check build log
		clGetProgramBuildInfo(program, device_id,CL_PROGRAM_BUILD_LOG, 0, NULL, &logSize);
		programLog = (char*)calloc(logSize + 1, sizeof(char));
		clGetProgramBuildInfo(program, device_id,CL_PROGRAM_BUILD_LOG, logSize + 1, programLog, NULL);
		printf("Build failed; error=%d, status=%d, programLog:nn%s",err, status, programLog);
		free(programLog);

		wbCheck(err);
	}

	// Create the compute kernel in the program we wish to run
	kernel = clCreateKernel(program, "vecAdd", &err);
	wbCheck(err);

	// Create the input and output arrays in device memory for our
	// calculation
	deviceInput1 = clCreateBuffer(context, CL_MEM_READ_ONLY,inputLengthBytes, NULL, NULL);
	deviceInput2 = clCreateBuffer(context, CL_MEM_READ_ONLY,inputLengthBytes, NULL, NULL);
	deviceOutput = clCreateBuffer(context, CL_MEM_WRITE_ONLY,inputLengthBytes, NULL, NULL);

	// Write our data set into the input array in device memory
	err = clEnqueueWriteBuffer(queue, deviceInput1, CL_TRUE, 0,inputLengthBytes, hostInput1, 0, NULL, NULL);
	err |= clEnqueueWriteBuffer(queue, deviceInput2, CL_TRUE, 0,inputLengthBytes, hostInput2, 0, NULL, NULL);
	wbCheck(err);

	// Set the arguments to our compute kernel
	err = clSetKernelArg(kernel, 0, sizeof(cl_mem), &deviceInput1);
	err |= clSetKernelArg(kernel, 1, sizeof(cl_mem), &deviceInput2);
	err |= clSetKernelArg(kernel, 2, sizeof(cl_mem), &deviceOutput);
	err |= clSetKernelArg(kernel, 3, sizeof(int), &inputLength);
	wbCheck(err);

	StartingTime = Time_start();
	// Execute the kernel over the entire range of the data set
	err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize,&localSize, 0, NULL, NULL);
	wbCheck(err);

	// Wait for the command queue to get serviced before reading back results
	wbCheck(clFinish(queue));
	Elapsed_time(StartingTime, "--------------->Kernel execution time ********", prt);
	// Read the results from the device
	wbCheck(clEnqueueReadBuffer(queue, deviceOutput, CL_TRUE, 0, inputLengthBytes,hostOutput, 0, NULL, NULL));

	/*Perform Success Test*/
	meanDiff=0.0;
		for (i = 0; i < inputLength; i++) {
		meanDiff = meanDiff + fabs(hostOutput[i] - hostExpected[i]);
	}
	meanDiff = meanDiff / (float)inputLength;
	if (meanDiff > 0.0001) {
		printf("Failed %f\n",meanDiff);

	}
	else {
		printf("%f Passed\n",meanDiff);
	}
	ExportRawFloat(argv4, hostOutput, inputLength);

	// release OpenCL resources
	clReleaseMemObject(deviceInput1);
	clReleaseMemObject(deviceInput2);
	clReleaseMemObject(deviceOutput);
	clReleaseProgram(program);
	clReleaseKernel(kernel);
	clReleaseCommandQueue(queue);
	clReleaseContext(context);

// release host memory
	free(hostInput1);
	free(hostInput2);
	free(hostOutput);
	free(hostExpected);
}

	return 1;
}


Overwriting lab10-OPENCL.cpp


**Compile OpenCL Code**

In [ ]:
!rm lab10-OPENCL.exe
!g++ lab10-OPENCL.cpp -o lab10-OPENCL.exe -lm -pthread -lOpenCL
!ls

Dataset  lab10-OPENCL.cpp  lab10-OPENCL.exe


**Execute OpenCL Code**

In [ ]:
!./lab10-OPENCL.exe



 +++++++++++++Test 0
Reading File Dataset/VectorAdd/Test/0/output.raw
Reading File Dataset/VectorAdd/Test/0/input0.raw
Reading File Dataset/VectorAdd/Test/0/input1.raw
The input length is 16
Importing data and creating memory on host Elapsed Time 0.108000 in mili-seconds
--------------->Kernel execution time ******** Elapsed Time 0.029000 in mili-seconds
0.000000 Passed
Writing File Dataset/VectorAdd/Test/0/Myoutput.raw


 +++++++++++++Test 1
Reading File Dataset/VectorAdd/Test/1/output.raw
Reading File Dataset/VectorAdd/Test/1/input0.raw
Reading File Dataset/VectorAdd/Test/1/input1.raw
The input length is 64
Importing data and creating memory on host Elapsed Time 0.137000 in mili-seconds
--------------->Kernel execution time ******** Elapsed Time 0.026000 in mili-seconds
0.000000 Passed
Writing File Dataset/VectorAdd/Test/1/Myoutput.raw


 +++++++++++++Test 2
Reading File Dataset/VectorAdd/Test/2/output.raw
Reading File Dataset/VectorAdd/Test/2/input0.raw
Reading File Dataset/Vector

**Save trace into Lab10-OPENCL.txt**

## **Also save your version of the notebook Lab10.ipynb **